In [ ]:
import time
import sys
import os
import numpy as np

%load_ext autoreload
%autoreload 2

import xfmkit.utils as utils
import xfmkit.argops as argops
import xfmkit.clustering as clustering
import xfmkit.visualisations as vis
import xfmkit.processops as processops
import xfmkit.structures as structures
import xfmkit.entry_processed as entry_processed

#add the parent folder to sys path so we can import from the notebook subfolder
sys.path.insert(0,'..')

#args = ' -d /home/lachlan/CODEBASE/xfmkit/data/processed_maps/carlos_full -n 2 -fc'
args_in = ["-d", "/home/lachlan/CODEBASE/xfmkit/data/processed_maps/carlos_full", "-fc" ]

PACKAGE_CONFIG='xfmkit/config.yaml'

In [ ]:
pxs, embedding, categories, classavg, palette = entry_processed.read_processed(args_in)

In [ ]:
i=11
print(pxs.labels)
print(pxs.labels[i])
print(pxs.weights[i])
np.mean(pxs.weighted.d[:,i]), np.max(pxs.weighted.d[:,i])

In [ ]:
fig = vis.tricolour_pixelset("Back", "AgL", "sum", pxs)